In [19]:
import pickle
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sklearn.preprocessing import StandardScaler
from datetime import date, datetime, timedelta

In [20]:
# Connect to the Laravel MySQL database
user = 'root'
password = ''
host = 'localhost'
database = 'sales_predictor'

# Create a SQLAlchemy engine
conn = create_engine(f'mysql+pymysql://{user}:{password}@{host}/{database}')

In [55]:
today = datetime.today().day

if today == 7:
    users = pd.read_sql('SELECT * from users', conn)

    for index in users.index:
        user_id = users.loc[index]['id']
        products = pd.read_sql("SELECT * from products where user_id='"+ str(user_id) + "'", conn)
        invoice_products = pd.read_sql("SELECT i.date AS date, ip.product_id AS product_id, ip.quantity AS unit FROM invoice_products ip JOIN invoices i ON ip.invoice_id = i.id where ip.user_id='"+ str(user_id) + "'", conn)

        invoice_products['date'] = pd.to_datetime(invoice_products['date'], format='%Y-%m-%d %I:%M:%S %p')
        invoice_products = invoice_products.sort_values(by = 'date')
        for i in invoice_products.index:
            print(invoice_products.loc[i]['date'])

2022-04-02 16:24:18
2022-04-02 16:24:18
2022-04-02 16:24:18
2022-04-02 16:24:18
2022-04-02 16:24:18
2022-04-02 16:24:18
2022-04-02 16:24:18
2022-04-02 16:24:18
2022-04-02 16:24:18
2022-04-02 16:24:18
2022-04-02 16:24:18
2022-04-02 16:24:18
2022-04-02 16:24:18
2022-04-02 16:24:18
2022-04-02 16:24:18
2022-04-03 19:19:13
2022-04-03 19:19:13
2022-04-03 19:19:13
2022-04-03 19:19:13
2022-04-03 19:19:13
2022-04-03 19:19:13
2022-04-03 19:19:13
2022-04-03 19:19:13
2022-04-04 15:34:38
2022-04-04 15:34:38
2022-04-04 15:34:38
2022-04-04 15:34:38
2022-04-04 15:34:38
2022-04-04 15:34:38
2022-04-04 15:34:38
2022-04-04 15:34:38
2022-04-04 15:34:38
2022-04-04 15:34:38
2022-04-04 15:34:38
2022-04-04 15:34:38
2022-04-05 21:38:13
2022-04-05 21:38:13
2022-04-05 21:38:13
2022-04-05 21:38:13
2022-04-05 21:38:13
2022-04-05 21:38:13
2022-04-05 21:38:13
2022-04-06 14:29:18
2022-04-06 14:29:18
2022-04-06 14:29:18
2022-04-06 14:29:18
2022-04-06 14:29:18
2022-04-06 14:29:18
2022-04-06 14:29:18
2022-04-06 14:29:18


In [52]:
# # Opening saved model
# with open("model/classifier.pkl", "rb") as file:
#     classifier = pickle.load(file)

# product_data = pd.read_csv("dataset/Project Dataset/product.csv")

# product_data = product_data.sort_values("product").reset_index().drop(columns=['index'])

# today = date.today()
# prediction_dates = [today + timedelta(days = i) for i in range(30)]

# sql = "INSERT INTO `predictions` (`id`, `date`, `product_id`, `unit`, `user_id`, `created_at`, `updated_at`) VALUES "

# for date in prediction_dates:
#     result = classifier.predict(pd.DataFrame({'day_of_year': date.timetuple().tm_yday, 'month':date.month, 'day_of_week':date.weekday()}, index = [0]))
#     i = 0
#     for value in result[0]:
#         if not value == 0:
#             sql += "(NULL, '" + str(date) + "', '" + str(product_data['id'][i]) + "', '" + str(value) + "', '5', CURRENT_TIMESTAMP, CURRENT_TIMESTAMP),"
#         i += 1
# print(sql)

INSERT INTO `predictions` (`id`, `date`, `product_id`, `unit`, `user_id`, `created_at`, `updated_at`) VALUES (NULL, '2024-09-07', '54', '2', '5', CURRENT_TIMESTAMP, CURRENT_TIMESTAMP),(NULL, '2024-09-07', '141', '1', '5', CURRENT_TIMESTAMP, CURRENT_TIMESTAMP),(NULL, '2024-09-07', '127', '1', '5', CURRENT_TIMESTAMP, CURRENT_TIMESTAMP),(NULL, '2024-09-07', '88', '1', '5', CURRENT_TIMESTAMP, CURRENT_TIMESTAMP),(NULL, '2024-09-07', '83', '1', '5', CURRENT_TIMESTAMP, CURRENT_TIMESTAMP),(NULL, '2024-09-07', '81', '1', '5', CURRENT_TIMESTAMP, CURRENT_TIMESTAMP),(NULL, '2024-09-07', '67', '1', '5', CURRENT_TIMESTAMP, CURRENT_TIMESTAMP),(NULL, '2024-09-08', '56', '1', '5', CURRENT_TIMESTAMP, CURRENT_TIMESTAMP),(NULL, '2024-09-08', '153', '1', '5', CURRENT_TIMESTAMP, CURRENT_TIMESTAMP),(NULL, '2024-09-08', '87', '1', '5', CURRENT_TIMESTAMP, CURRENT_TIMESTAMP),(NULL, '2024-09-08', '133', '1', '5', CURRENT_TIMESTAMP, CURRENT_TIMESTAMP),(NULL, '2024-09-08', '88', '1', '5', CURRENT_TIMESTAMP, CURREN